## Build a complete table of metadata and experimental details

As originally defined, using only with isolation source and collection data, there exist duplicate rows. 

These duplicates were intended to record multiple extractions from the same sample due to problems. (technical replicates)

However, source and date are still the same and it is only when joined to the sequencing run table that these additional extractions take on meaning.

Below we join the tables and then add a replicate_number column to disambiguate.

Mock and negative control records complicate matters.

A unique index (sample_name) is constructed sysmetically from isolation_source, collection_date and replicate_number.


In [1]:
import pandas as pd
import numpy as np 

In [2]:
# read in the sample metadata table as it current exists.

samples = pd.read_excel('../source_data/pigs_samples_IDs_for_NCBI.xlsx', sheet_name=0, skiprows=14, skipfooter=0, index_col=None, dtype={'*sample_name': str})
print('There were {} rows in sample table'.format(len(samples)))
samples.head()



There were 885 rows in sample table


,*sample_name,sample_title,bioproject_accession,*organism,*collection_date,*env_biome,*env_feature,*env_material,*geo_loc_name,*host,...,perturbation,samp_collect_device,samp_mat_process,samp_size,samp_store_temp,samp_vol_we_dna_ext,description,PigPen,Cohort,Dysentery
0,Ja31/14286,NaN,NaN,Gut microflora metagenomics,2017-01-31,Host- gut,Faecal,Faecal,"NSW, Australia",Porcine,...,NaN,NaN,NaN,NaN,-80C,NaN,NaN,1A,NaN,0.0
1,Ja31/29962,NaN,NaN,Gut microflora metagenomics,2017-01-31,Host- gut,Faecal,Faecal,"NSW, Australia",Porcine,...,NaN,NaN,NaN,NaN,-80C,NaN,NaN,1A,NaN,0.0
2,Ja31/29707,NaN,NaN,Gut microflora metagenomics,2017-01-31,Host- gut,Faecal,Faecal,"NSW, Australia",Porcine,...,NaN,NaN,NaN,NaN,-80C,NaN,NaN,1A,NaN,0.0
3,Ja31/14322,NaN,NaN,Gut microflora metagenomics,2017-01-31,Host- gut,Faecal,Faecal,"NSW, Australia",Porcine,...,NaN,NaN,NaN,NaN,-80C,NaN,NaN,1A,NaN,0.0
4,Ja31/29913,NaN,NaN,Gut microflora metagenomics,2017-01-31,Host- gut,Faecal,Faecal,"NSW, Australia",Porcine,...,NaN,NaN,NaN,NaN,-80C,NaN,NaN,1A,NaN,0.0


In [3]:
pd.options.display.max_rows = 10
seqruns = pd.read_excel('../source_data/DNA_plates.xlsx', sheet_name=0, index_col=None, dtype={'date and pig ID': str})
seqruns.drop(seqruns.columns[[3,5]], axis=1, inplace=True)
print('There were {} rows in the DNA pate table'.format(len(seqruns)))
seqruns.head()

There were 928 rows in the DNA pate table


,elution plate number,well position,date of sample,pig ID,date and pig ID
0,P1,A1,Fe21,14194,Fe21/14194
1,P1,A2,Fe28,14286,Fe28/14286
2,P1,A3,Fe21,29644,Fe21/29644
3,P1,A4,Fe14,29898,Fe14/29898
4,P1,A5,Fe21,29679,Fe21/29679


In [4]:
def demux(df):
    # replace the current sample_name with that made from isolation_source and collection_date
    df['alt_sample_name'] = df['isolation_source'].astype(str) + df['*collection_date'].map(lambda v: v.strftime('-%y%m%d') if not pd.isnull(v) else '') 

    # + "-" + ncbi.replicate_number.astype(str)

    def f(n):
        """
        Create a incrementing counter of replicates when encountering duplicated rows.
        """
        if len(n) > 1:
           print('{} was duplicated {} times'.format(n.name, len(n)))
        return pd.Series(np.arange(1, len(n)+1), index=n.index)

    # add a new column which records the number of replicates of a given isolation source and collection date
    df['replicate'] = 0
    df.replicate = df.groupby('alt_sample_name')['alt_sample_name'].apply(f)
    df['alt_sample_name'] = df['isolation_source'].astype(str) + df['*collection_date'].map(lambda v: v.strftime('-%y%m%d') if not pd.isnull(v) else '') + "-" + df.replicate.map('{:02d}'.format)

In [66]:
def demux2(df):
    # replace the current sample_name with that made from isolation_source and collection_date
    df['sn'] =  df['*collection_date'].map(lambda v: v.strftime('%b%d/') if not pd.isnull(v) else 'NA/') + df['isolation_source'].astype(str)

    def f(n):
        """
        Create a incrementing counter of replicates when encountering duplicated rows.
        """
        if len(n) > 1:
           print('{} was duplicated {} times'.format(n.name, len(n)))
        return pd.Series(np.arange(1, len(n)+1), index=n.index)

    # add a new column which records the number of replicates of a given isolation source and collection date
    df['replicate'] = 0
    df.replicate = df.groupby('sn')['sn'].apply(f)
    df['sn'] = df['*collection_date'].map(lambda v: v.strftime('%b%d/') if not pd.isnull(v) else 'NA/') + df['isolation_source'].astype(str)  + "-" + df.replicate.map('{:02d}'.format)

In [5]:
# Using the existing sample_name as an index, join samples and plates. 
# rebuild an integer index
cmb = samples.set_index('*sample_name',).join(seqruns.set_index('date and pig ID'), how='inner').reset_index()
# given the old sample name column a better name than 'index'
cmb.rename(columns={'index': 'old_sample_name'}, inplace=True)
print('There were {} rows in the joined table'.format(len(cmb)))

# Only one record should exist for a given plate and well.
cmb.drop_duplicates(['elution plate number', 'well position'], inplace=True)
print('There were {} rows after removal of plate/well duplicates'.format(len(cmb)))

demux(cmb)
cmb.to_csv('test.csv')


There were 922 rows in the joined table
There were 911 rows after removal of plate/well duplicates
14160-170201 was duplicated 2 times
14162-170201 was duplicated 2 times
14172-170216 was duplicated 2 times
14174-170131 was duplicated 2 times
14190-170131 was duplicated 3 times
14192-170131 was duplicated 2 times
14193-170131 was duplicated 2 times
14194-170207 was duplicated 2 times
14263-170131 was duplicated 2 times
14265-170131 was duplicated 2 times
14274-170131 was duplicated 2 times
14284-170131 was duplicated 2 times
14298-170131 was duplicated 2 times
14320-170131 was duplicated 2 times
29645-170131 was duplicated 2 times
29652-170131 was duplicated 2 times
29653-170131 was duplicated 2 times
29667-170131 was duplicated 2 times
29668-170131 was duplicated 2 times
29694-170131 was duplicated 2 times
29718-170131 was duplicated 2 times
29743-170131 was duplicated 2 times
29753-170131 was duplicated 2 times
29754-170131 was duplicated 2 times
29781-170131 was duplicated 2 times
2

In [6]:
pd.options.display.max_rows = 5
cmb

,old_sample_name,sample_title,bioproject_accession,*organism,*collection_date,*env_biome,*env_feature,*env_material,*geo_loc_name,*host,...,description,PigPen,Cohort,Dysentery,elution plate number,well position,date of sample,pig ID,alt_sample_name,replicate
0,18Ja24/ColiGuard_P03,NaN,NaN,DPI,2018-01-24,Lab,Probiotic ColiGuard,Probiotic ColiGuard,"NSW, Australia",NaN,...,NaN,NaN,NaN,NaN,P3,H1,18Ja24,ColiGuard_P3,ColiGuard-180124-01,1
1,18Ja24/ColiGuard_P04,NaN,NaN,DPI,2018-01-24,Lab,Probiotic ColiGuard,Probiotic ColiGuard,"NSW, Australia",NaN,...,NaN,NaN,NaN,NaN,P4,D3,18Ja24,ColiGuard_P4,ColiGuard-180124-02,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,NA/neg.control_P10,NaN,NaN,Laboratory - UTS,NaT,Lab,negative control,negative control,"NSW, Australia",NaN,...,NaN,NaN,NaN,NaN,P10,D10,NaN,neg.control_P10,NegativeControl-18,18
921,NA/neg.control_P10.2,NaN,NaN,Laboratory - UTS,NaT,Lab,negative control,negative control,"NSW, Australia",NaN,...,NaN,NaN,NaN,NaN,P10,G6,NaN,neg.control_P10.2,NegativeControl-19,19


In [8]:
files = pd.read_csv('../fq_well_and_plate.txt', sep=' ', names=['DNA_plate','DNA_well','filename'], index_col=['DNA_plate', 'DNA_well'])
# we will convert this to one row per read-pair, rather than one row for R1 and one for R2
files = files.groupby(['DNA_plate', 'DNA_well']).agg(['first', 'last'])
# convert the pandas group to a new dataframe
files = pd.DataFrame(files.reset_index().values, columns=['DNA_plate', 'DNA_well', 'r1_filename', 'r2_filename'])
print('There were {} file pairs'.format(len(files)))
files.head()

There were 960 file pairs


,DNA_plate,DNA_well,r1_filename,r2_filename
0,P1,A1,MON5838/MON5838_1/plate_1_A1_S1_R1_001.fastq.gz,MON5838/MON5838_1/plate_1_A1_S1_R2_001.fastq.gz
1,P1,A10,MON5838/MON5838_73/plate_1_A10_S73_R1_001.fast...,MON5838/MON5838_73/plate_1_A10_S73_R2_001.fast...
2,P1,A11,MON5838/MON5838_81/plate_1_A11_S81_R1_001.fast...,MON5838/MON5838_81/plate_1_A11_S81_R2_001.fast...
3,P1,A12,MON5838/MON5838_89/plate_1_A12_S89_R1_001.fast...,MON5838/MON5838_89/plate_1_A12_S89_R2_001.fast...
4,P1,A2,MON5838/MON5838_9/plate_1_A2_S9_R1_001.fastq.gz,MON5838/MON5838_9/plate_1_A2_S9_R2_001.fastq.gz


In [10]:
cmb.rename(columns={'elution plate number': 'DNA_plate', 'well position': 'DNA_well'}, inplace=True)
out = cmb.set_index(['DNA_plate', 'DNA_well']).join(files.set_index(['DNA_plate', 'DNA_well']))

In [11]:
out.to_csv('out.csv')